In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 球種選擇

In [ ]:
hittype = 0 # 高遠球:0, 挑球:1

if hittype == 0:
  timeLenth = 60
  hittype_name = "high"

elif hittype == 1:
  timeLenth = 50
  hittype_name = "short"

else:
  print("error hittype")


# cut function

In [ ]:
import os
import numpy as np
import csv

def slope(input_data):
    slope_col = np.zeros(len(input_data),dtype=float)
    for i in range(len(input_data) - 1):
        for j in range(1,4):
            temp = input_data[i+1][j] - input_data[i][j]
            slope_col[i] += temp**2
        slope_col[i] = slope_col[i]**0.5
    return slope_col

def ma(slope_col, n):
    slope_ma = np.zeros(len(slope_col),dtype=float)
    for i in range(n,len(slope_col)-n):
        for j in range(-n,n+1):
            slope_ma[i] += slope_col[i+j]
        slope_ma[i] /= float(2*n+1)
    return slope_ma

def data_cut(input_data, save_data, hit_type):
    slope_col = slope(input_data)
    slope_avg = np.average(slope_col)
    # n = 4
    if hit_type == "high":
        n = 6
    elif hit_type == "short":
        n = 5
    slope_ma = ma(slope_col, n)

    for i in range(60,len(slope_col)-50):

        if ( slope_ma[i] > slope_avg ) and ( slope_col[i]==max(slope_col[i-50:i+50]) ) : # 找到可能峰值
            start = 0  # 向前&向後找起點
            end = 0
            while i+start > (50+n) :
                start -= 1
                if slope_ma[i+start] <= slope_avg:
                    break
            while i+end < (len(slope_col)-50-n) :
                end += 1
                if slope_ma[i+end] <= slope_avg:
                    break

            if  hit_type == "high" and end-start > 40 and slope_ma[i] > 15: # 長遠球
                save_data.append(input_data[i-45:i+15, [1,2,3,5,6,7]])

            elif hit_type == "short" and end-start > 20 and slope_ma[i] > 15: # 挑球
                save_data.append(input_data[i-35:i+15, [1,2,3,5,6,7]])

# Removes null bytes from the input file and returns a sanitized version of the file.
def sanitize_file(input_file_path):
    sanitized_content = ""
    with open(input_file_path, 'r', encoding='utf-8', errors='replace') as f:
        content = f.read()
        sanitized_content = content.replace('\x00', '')

    return sanitized_content

In [ ]:
# 指定資料集所在的路徑
data_path =  "/content/drive/MyDrive/資科羽球專題/data(.txt)"

# 取得該路徑下所有的檔案名稱
all_files = os.listdir(data_path)

# 過濾出所有的 .txt 檔案
txt_files = [file_name for file_name in all_files if file_name.endswith('.txt') and hittype_name in file_name and len(file_name)>=30]

labels = np.empty((0, 5), dtype=int)
data = np.empty((0, timeLenth, 6), dtype=float)

for t, input_filename in enumerate(txt_files):

    print(input_filename)

    input_file_path = os.path.join(data_path, input_filename)

    # Sanitize the file by removing null bytes
    sanitized_content = sanitize_file(input_file_path)

    lines = sanitized_content.split('\n')

    # Starting from the last line, move upwards until a complete line (with 7 commas) is found
    while lines and lines[-1].count(",") != 7:
        lines = lines[:-1]

    # Load the (potentially modified) data into a numpy array
    input_data = np.loadtxt(lines, delimiter=",", dtype=float)

    save_data = []

    data_cut(input_data, save_data, hittype_name)

    save_data = np.array(save_data).astype(float)

    data = np.concatenate((data, save_data), axis=0)

    label_values = np.array([int(ch) for ch in input_filename[6:11]])  # 轉換為整數陣列

    label_values = label_values[np.newaxis, :]  # 增加一個維度以使其成為二維陣列

    for i in range(len(save_data)):
        labels = np.vstack((labels, label_values))

print(data.shape)
print(labels.shape)

h01_n_23342_110358030_high3.txt
h02_n_11122_101108044_high.txt
h03_y_55434_109703024_high.txt
h05_y_11111_110703043_high.txt
h06_y_33222_110302059_high.txt
h07_y_23221_109703056_high.txt
h08_y_22111_109703034_high.txt
h10_y_24244_109703003_high.txt
h11_n_22122_109540020_high.txt
h12_y_45543_109507014_high.txt
h13_y_11111_109305061_high.txt
h14_y_22121_109206049_high.txt
h15_y_33321_109101023_high.txt
h16_y_22111_109703018_high.txt
h09_y_23122_109703019_high.txt
(406, 60, 6)
(406, 5)


In [ ]:
# 指定資料集所在的路徑
data_path =  "/content/drive/MyDrive/資科羽球專題/data(.txt)/1026"

# 取得該路徑下所有的檔案名稱
all_files = os.listdir(data_path)

# 過濾出所有的 .txt 檔案
txt_files = [file_name for file_name in all_files if file_name.endswith('.txt') and hittype_name in file_name and len(file_name)>=30]

for t, input_filename in enumerate(txt_files):

    print(input_filename)

    input_file_path = os.path.join(data_path, input_filename)

    # Sanitize the file by removing null bytes
    sanitized_content = sanitize_file(input_file_path)

    lines = sanitized_content.split('\n')

    # Starting from the last line, move upwards until a complete line (with 7 commas) is found
    while lines and lines[-1].count(",") != 7:
        lines = lines[:-1]

    # Load the (potentially modified) data into a numpy array
    input_data = np.loadtxt(lines, delimiter=",", dtype=float)

    save_data = []

    data_cut(input_data, save_data, hittype_name)

    save_data = np.array(save_data).astype(float)

    data = np.concatenate((data, save_data), axis=0)

    label_values = np.array([int(ch) for ch in input_filename[6:11]])  # 轉換為整數陣列

    label_values = label_values[np.newaxis, :]  # 增加一個維度以使其成為二維陣列

    for i in range(len(save_data)):
        labels = np.vstack((labels, label_values))

# 指定資料集所在的路徑
data_path =  "/content/drive/MyDrive/資科羽球專題/data(.txt)/1031"

# 取得該路徑下所有的檔案名稱
all_files = os.listdir(data_path)

# 過濾出所有的 .txt 檔案
txt_files = [file_name for file_name in all_files if file_name.endswith('.txt') and hittype_name in file_name and len(file_name)>=30]

for t, input_filename in enumerate(txt_files):

    print(input_filename)

    input_file_path = os.path.join(data_path, input_filename)

    # Sanitize the file by removing null bytes
    sanitized_content = sanitize_file(input_file_path)

    lines = sanitized_content.split('\n')

    # Starting from the last line, move upwards until a complete line (with 7 commas) is found
    while lines and lines[-1].count(",") != 7:
        lines = lines[:-1]

    # Load the (potentially modified) data into a numpy array
    input_data = np.loadtxt(lines, delimiter=",", dtype=float)

    save_data = []

    data_cut(input_data, save_data, hittype_name)

    save_data = np.array(save_data).astype(float)

    data = np.concatenate((data, save_data), axis=0)

    label_values = np.array([int(ch) for ch in input_filename[6:11]])  # 轉換為整數陣列

    label_values = label_values[np.newaxis, :]  # 增加一個維度以使其成為二維陣列

    for i in range(len(save_data)):
        labels = np.vstack((labels, label_values))

print(data.shape)
print(labels.shape)

h08_y_23233_102611108_high.txt
h07_y_12112_102611107_high.txt
h09_y_34432_102611109_high.txt
h10_y_12111_102611110_high.txt
h11_y_12122_102611111_high.txt
h12_y_22111_102611112_high.txt
h13_y_23121_102611113_high.txt
h01_y_55555_102611101_high.txt
h02_y_45554_102611102_high.txt
h03_y_55555_102611103_high.txt
h04_y_55555_102611104_high.txt
h05_y_55555_102611105_high.txt
h06_y_55444_102611106_high.txt
h11_y_55555_103100011_high.txt
h09_y_55555_103100009_high.txt
h08_y_55554_103100008_high.txt
h07_y_55555_103100007_high.txt
h06_y_55545_103100006_high.txt
h05_y_55555_103100005_high.txt
h10_y_55555_103100010_high.txt
h01_y_55555_103100001_high.txt
h02_y_44444_103100002_high.txt
h03_y_22233_103100003_high.txt
h04_y_55454_103100004_high.txt
(848, 60, 6)
(848, 5)


# Training

In [ ]:
from sklearn.model_selection import train_test_split
#定義訓練集和測試集的比例，例如訓練集佔 80%，測試集佔 20%
train_ratio = 0.8
test_ratio = 0.2
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=test_ratio, random_state=24)

print(x_train.shape)
print(x_test.shape)

(678, 60, 6)
(170, 60, 6)


In [ ]:
x_train = x_train.reshape(x_train.shape)
x_test = x_test.reshape(x_test.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# 自定義輸出函數
def custom_output(x):
    return x

# 自定義損失函數
def custom_loss(y_true, y_pred):
    loss = torch.mean((y_true - y_pred) ** 2)
    # loss = torch.mean(torch.square(y_true - y_pred))
    return loss

# 自定義準確性函數
def custom_accuracy(y_true, y_pred):
    return torch.mean((torch.abs(y_true - y_pred) < 0.5).float(), dtype=torch.float32)

# LSTM
class CustomLSTMModel(nn.Module):
    def __init__(self, input_size, units, output_dim, num_layers, dropout_rate):
        super(CustomLSTMModel, self).__init__()
        self.lstm_layers = nn.ModuleList(
            [nn.LSTM(input_size if i == 0 else units, units, batch_first=True) for i in range(num_layers)]
        )
        self.dropout = nn.Dropout(p=dropout_rate)
        self.dense = nn.Linear(units, output_dim)

    def forward(self, x):
        for lstm_layer in self.lstm_layers:
            x, _ = lstm_layer(x)
            x = self.dropout(x)
        x = x[:, -1, :]  # 只取序列的最後一個時間步
        x = self.dense(x)
        return custom_output(x)

# GRU
class CustomGRUModel(nn.Module):
    def __init__(self, input_size, units, output_dim, num_layers, dropout_rate):
        super(CustomGRUModel, self).__init__()
        self.gru_layers = nn.ModuleList(
            [nn.GRU(input_size if i == 0 else units, units, batch_first=True) for i in range(num_layers)]
        )
        self.dropout = nn.Dropout(p=dropout_rate)
        self.dense = nn.Linear(units, output_dim)

    def forward(self, x):
        for gru_layer in self.gru_layers:
            x, _ = gru_layer(x)
            x = self.dropout(x)
        x = x[:, -1, :]  # 只取序列的最後一個時間步
        x = self.dense(x)
        return custom_output(x)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [ ]:
# Early Stopping 早停法
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        """
        Early stopping utility.

        :param patience: 數量的 epochs，在這些 epochs 中如果損失沒有改善，則提前停止訓練。
        :param min_delta: 被認為是改善的最小變化量。
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
# 超參數

learning_rate = 0.01
batch_size = 20
numEpoch = 40
output_dim = 5
num_layers = 2
dropout_rate = 0.1
input_size = 6

### LSTM Result

In [ ]:
# 使用MyDataset類
train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)

# 使用DataLoader來創建數據集對象
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Instantiate the custom model
model = CustomLSTMModel(input_size=input_size, units=64, output_dim=output_dim, num_layers=num_layers, dropout_rate=dropout_rate)

# 定義損失函數和優化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = custom_loss

# 訓練循環
early_stopping = EarlyStopping(patience=10, min_delta=0.01)

for epoch in range(numEpoch):
    model.train()
    for x_batch, y_batch in train_dataloader:
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = loss_fn(y_batch, y_pred)
        loss.backward()
        optimizer.step()

    # 評估
    model.eval()
    with torch.no_grad():
        total_loss = 0
        total_accuracy = 0
        for x_batch, y_batch in test_dataloader:
            y_pred = model(x_batch)
            total_loss += loss_fn(y_batch, y_pred).item()
            total_accuracy += custom_accuracy(y_batch, y_pred).item()
    avg_loss = total_loss / len(test_dataloader)
    avg_accuracy = total_accuracy / len(test_dataloader)
    print(f'Epoch {epoch+1}, Loss: {avg_loss}, Accuracy: {avg_accuracy}')

    # Early Stopping 檢查
    early_stopping(avg_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

Epoch 1, Loss: 1.74037770430247, Accuracy: 0.2622222188446257
Epoch 2, Loss: 1.4938281377156575, Accuracy: 0.2466666665342119
Epoch 3, Loss: 1.4506947331958346, Accuracy: 0.2822222196393543
Epoch 4, Loss: 1.4208719664149814, Accuracy: 0.3099999974171321
Epoch 5, Loss: 1.1966414716508653, Accuracy: 0.4444444411330753
Epoch 6, Loss: 1.0938887496789296, Accuracy: 0.465555555290646
Epoch 7, Loss: 0.90241054031584, Accuracy: 0.4666666587193807
Epoch 8, Loss: 0.7573856628603406, Accuracy: 0.4933333231343163
Epoch 9, Loss: 0.6362450685766008, Accuracy: 0.5122222205003103
Epoch 10, Loss: 0.7548021309905582, Accuracy: 0.5299999978807237
Epoch 11, Loss: 0.6490569611390432, Accuracy: 0.573333332935969
Epoch 12, Loss: 0.6856697681877348, Accuracy: 0.5444444384839799
Epoch 13, Loss: 0.6715064860052533, Accuracy: 0.5622222191757626
Epoch 14, Loss: 0.6966277029779222, Accuracy: 0.5677777760558658
Epoch 15, Loss: 0.7265311843819089, Accuracy: 0.5444444384839799
Epoch 16, Loss: 0.8304359846644931, Accu

### GRU Result

In [ ]:
# 使用MyDataset類
train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)

# 使用DataLoader來創建數據集對象
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Instantiate the custom model
model = CustomGRUModel(input_size=input_size, units=64, output_dim=output_dim, num_layers=num_layers, dropout_rate=dropout_rate)

# 定義損失函數和優化器
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = custom_loss

# 訓練循環
early_stopping = EarlyStopping(patience=10, min_delta=0.01)

for epoch in range(numEpoch):
    model.train()
    for x_batch, y_batch in train_dataloader:
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = loss_fn(y_batch, y_pred)
        loss.backward()
        optimizer.step()

    # 評估
    model.eval()
    with torch.no_grad():
        total_loss = 0
        total_accuracy = 0
        for x_batch, y_batch in test_dataloader:
            y_pred = model(x_batch)
            total_loss += loss_fn(y_batch, y_pred).item()
            total_accuracy += custom_accuracy(y_batch, y_pred).item()
    avg_loss = total_loss / len(test_dataloader)
    avg_accuracy = total_accuracy / len(test_dataloader)
    print(f'Epoch {epoch+1}, Loss: {avg_loss}, Accuracy: {avg_accuracy}')

    # Early Stopping 檢查
    early_stopping(avg_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

Epoch 1, Loss: 1.1648907661437988, Accuracy: 0.3122222257985009
Epoch 2, Loss: 0.8109479579660628, Accuracy: 0.4911111063427395
Epoch 3, Loss: 0.6279211706585355, Accuracy: 0.5377777748637729
Epoch 4, Loss: 0.6942077941364713, Accuracy: 0.5677777694331275
Epoch 5, Loss: 0.4506952183114158, Accuracy: 0.6766666637526618
Epoch 6, Loss: 0.49642591012848747, Accuracy: 0.6800000071525574
Epoch 7, Loss: 0.46355507771174115, Accuracy: 0.7311111092567444
Epoch 8, Loss: 0.3929712093538708, Accuracy: 0.694444457689921
Epoch 9, Loss: 0.24266531566778818, Accuracy: 0.7677777740690443
Epoch 10, Loss: 0.3751177423530155, Accuracy: 0.7855555613835653
Epoch 11, Loss: 0.3460340549548467, Accuracy: 0.8088888857099745
Epoch 12, Loss: 0.37087380554940963, Accuracy: 0.817777779367235
Epoch 13, Loss: 0.3505641304784351, Accuracy: 0.8499999973509047
Epoch 14, Loss: 0.3559948297010528, Accuracy: 0.8066666656070285
Epoch 15, Loss: 0.3218233154879676, Accuracy: 0.8100000023841858
Epoch 16, Loss: 0.28457086947229